In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1)
plt.style.use('ggplot')

import pandas as pd
import numpy as np
import random as rnd

from sklearn.cross_validation import KFold, cross_val_score

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier



train=pd.read_csv('../train_abuse.csv')
validate=pd.read_csv('../validate_abuse.csv')
#test=pd.read_csv('../testwDSM.csv')

train.shape, validate.shape, #test.shape

((325514, 33), (160328, 33))

In [16]:
print train['DSMCRIT'].value_counts() / train['DSMCRIT'].sum()


5     0.040132
8     0.034516
4     0.025692
7     0.014277
6     0.010950
2     0.004777
9     0.004515
10    0.004201
3     0.003735
11    0.002191
13    0.001716
20    0.001521
0     0.001431
12    0.001394
15    0.001227
17    0.000774
1     0.000677
19    0.000582
14    0.000458
16    0.000372
18    0.000121
Name: DSMCRIT, dtype: float64


In [18]:
#train.describe()
train = train.sample(20000)
validate = validate.sample(6000)
train.shape, validate.shape, validate.head(2)

((20000, 33),
 (6000, 33),
         AGE  GENDER  RACE  ETHNIC  MARSTAT  EDUC  EMPLOY  DETNLF  PREG  VET  \
 28929     3       1     4       5        1     3       4       2     2    2   
 147657    5       2     5       5        1     2       4       5     2    2   
 
          ...     FREQ2  FRSTUSE2  SUB3  ROUTE3  FREQ3  FRSTUSE3  NUMSUBS  IDU  \
 28929    ...         1         2     3      20      2         3        3    0   
 147657   ...         1         2     4       2      1         2        3    0   
 
         DSMCRIT  PSYPROB  
 28929        13        2  
 147657        6        1  
 
 [2 rows x 33 columns])

In [19]:
#train = train.query('SUB1 <= 10').query('SUB2 <= 10')
#validate = validate.query('SUB1 <= 10').query('SUB2 <= 10')

drop_list = ['DSMCRIT',  #'NUMSUBS'
             ]

X_train = train.drop(drop_list, axis=1)
Y_train = train["DSMCRIT"]
X_validate = validate.drop(drop_list, axis=1)
Y_validate = validate["DSMCRIT"]
#X_test  = test.drop(drop_list, axis=1)
X_train.shape, X_validate.shape, #X_test.shape



((20000, 32), (6000, 32))

In [20]:
print X_train.columns.tolist()

['AGE', 'GENDER', 'RACE', 'ETHNIC', 'MARSTAT', 'EDUC', 'EMPLOY', 'DETNLF', 'PREG', 'VET', 'LIVARAG', 'ARRESTS', 'SERVSETA', 'METHUSE', 'PSOURCE', 'DETCRIM', 'NOPRIOR', 'SUB1', 'ROUTE1', 'FREQ1', 'FRSTUSE1', 'SUB2', 'ROUTE2', 'FREQ2', 'FRSTUSE2', 'SUB3', 'ROUTE3', 'FREQ3', 'FRSTUSE3', 'NUMSUBS', 'IDU', 'PSYPROB']


In [21]:
#one hot
from sklearn import preprocessing

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(X_train)

# 3. Transform
onehotlabels = enc.transform(X_train).toarray()
#onehotlabels.shape
X_train = onehotlabels

onehotlabels = enc.transform(X_validate).toarray()
X_validate = onehotlabels

X_train.shape, X_validate.shape

((20000, 214), (6000, 214))

In [22]:
# Logistic Regression
logreg = LogisticRegression(n_jobs=-1)
logreg.fit(X_train, Y_train)
#Y_pred = logreg.predict(X_test)

l_acc_log = cross_val_score(logreg, X_train, Y_train, cv=5)
acc_log = round(np.mean(l_acc_log), 3)

l_acc_log = ['%.3f' % elem for elem in l_acc_log]
print l_acc_log
print acc_log

['0.636', '0.635', '0.636', '0.641', '0.640']
0.638


In [23]:
# Random Forest (slow)

random_forest = RandomForestClassifier(n_estimators=200, max_depth=20, n_jobs=-1)
random_forest.fit(X_train, Y_train)
#Y_pred = random_forest.predict(X_test)

l_acc_random_forest = cross_val_score(random_forest, X_train, Y_train, cv=5)
acc_random_forest = round(np.mean(l_acc_random_forest), 3)

l_acc_random_forest = ['%.3f' % elem for elem in l_acc_random_forest]
print l_acc_random_forest
print acc_random_forest

['0.631', '0.639', '0.630', '0.635', '0.641']
0.635


In [24]:
# Linear SVC
linear_svc = LinearSVC(C=0.001)
linear_svc.fit(X_train, Y_train)
#Y_pred = linear_svc.predict(X_test)

l_acc_linear_svc = cross_val_score(linear_svc, X_train, Y_train, cv=5)
acc_linear_svc = round(np.mean(l_acc_linear_svc), 3)
l_acc_linear_svc = ['%.3f' % elem for elem in l_acc_linear_svc]
print l_acc_linear_svc
print acc_linear_svc

['0.621', '0.624', '0.621', '0.626', '0.628']
0.624


In [25]:
print 'week8-newsplit-score'
models = pd.DataFrame({
    'Model': ['Logistic Regression', 
              'Random Forest','Linear SVC'],
    'Cross Validation': [l_acc_log, 
              l_acc_random_forest, l_acc_linear_svc],
    'Cross Validation Mean': [acc_log, 
              acc_random_forest, acc_linear_svc]
    })
models.sort_values(by='Cross Validation Mean', ascending=False)


week8-newsplit-score


,Cross Validation,Cross Validation Mean,Model
0,"[0.636, 0.635, 0.636, 0.641, 0.640]",0.638,Logistic Regression
1,"[0.631, 0.639, 0.630, 0.635, 0.641]",0.635,Random Forest
2,"[0.621, 0.624, 0.621, 0.626, 0.628]",0.624,Linear SVC
